### Callbacks

In [3]:
function callback() {
    setTimeout(() => console.log('callback two'), 3000)
    console.log('In callback')
}

In [5]:
let x

In [6]:
x = 5
setTimeout(callback, 5000)

Timeout {
  _idleTimeout: 5000,
  _idlePrev: [TimersList],
  _idleNext: [TimersList],
  _idleStart: 309699,
  _onTimeout: [Function: callback],
  _timerArgs: undefined,
  _repeat: null,
  _destroyed: false,
  [Symbol(refed)]: true,
  [Symbol(asyncId)]: 54,
  [Symbol(triggerId)]: 51
}

In callback
callback two
